In [48]:
import pandas as pd
pd.set_option('display.max_columns', None)

def get_df():
    sale_path = "data/SALE JAN FINAL 2024.XLS"
    df = pd.read_excel(sale_path, sheet_name="MARG ERP 9+ Excel Report", header=None)
    return df

In [17]:
df = get_df()
print(df.shape)
df.head()

(2612, 14)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,IKSHANA MEDICAL AND SURGICAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"1ST FLO KOHLI TOWERS THANDI ROAD CIVIL LINES,H...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Phone : 9058591180 E-Mail : IKSHANAMEDICALANDS...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,SALES-TAX ACCOUNT REGISTER 01/01/2024 - 31/01/...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BILL NO.,PARTY NAME,GST.No.,BILL VALUE,[------------LS-GST,5,%-,-----------],[-----------LS-GST 12,NaN,%-----------],[-----------LS-GST 18,%-----------],OTHERS


In [20]:
df1 = df.iloc[6:, :]
print(df1.shape)
df1.head()

(2606, 14)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13
6,02/01/2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,A013054,SAIFI,NaN,204,90.48,2.26,2.26,97.63,5.86,5.86,0,0,0,-0.35
8,A013055,ARCHANA SHARMA,NaN,130,0,0,0,116.08,6.96,6.96,0,0,0,0
9,A013056,RAMAUTAR SHARMA,NaN,169,0,0,0,150.9,9.05,9.05,0,0,0,0
10,A013057,DINESH KUMAR TEWARI,NaN,1030,0,0,0,776.78,46.61,46.61,135.6,12.2,12.2,0


In [ ]:
columns = [
    "BILL NO.",
    "PARTY NAME",
    "GST.No.",
    "BILL VALUE",
    "[------------LS-GST",
    "5",
    "%-",
    "-----------]",
    "[-----------LS-GST 12",
    "",
    "%-----------]",
    "[-----------LS-GST 18",
    "%-----------]",
    "OTHERS",
]

In [99]:
# Initialize variables to keep track of current bill date and bill number
current_bill_date = None
current_bill_no = None

result_df = pd.DataFrame(
    columns=[
        "Inv No.",
        "Inv Date",
        "Party/Cash",
        "Product's Name",
        "HSN Code",
        "Qty",
        "Rate",
        "GST%",
        "Amount",
        "CGST %",
        "CGST Amt",
        "SGST %",
        "SGST Amt",
        "IGST Amt",
        "Discount %",
        "Total",
        "State",
        "Party's GST",
        "Reg Type",
        "Place of Supply",
        "Country",
        "Consignee State",
        "Consignee GST",
    ]
)


def get_transformed_rows(row, bill_date, calculate_igst=False):
    # Extract required values from the row
    inv_no = row.iloc[0]
    party_or_cash = row.iloc[1]
    amount_5 = row.iloc[4]
    amount_12 = row.loc[7]
    amount_18 = row.loc[10]

    new_rows = []
    for per, amount in [(5, amount_5), (12, amount_12), (18, amount_18)]:
        cgst_per = per / 2
        sgst_per = per / 2
        cgst_amt = amount * (cgst_per / 100)
        sgst_amt = amount * (sgst_per / 100)
        igst_amt = amount * (per / 100) if calculate_igst else 0.0
        discount = 0.0

        new_row = {
            "Inv No.": inv_no,
            "Inv Date": bill_date,
            "Party/Cash": party_or_cash,
            "Product's Name": "",
            "HSN Code": "",
            "Qty": "",
            "Rate": "",
            "GST%": per,
            "Amount": amount,
            "CGST %": cgst_per,
            "CGST Amt": cgst_amt,
            "SGST %": sgst_per,
            "SGST Amt": sgst_amt,
            "IGST Amt": igst_amt,
            "Discount %": discount,
            "Total": amount + cgst_amt + sgst_amt + igst_amt - discount,
            "State": "Uttarakhand",
            "Party's GST": "",
            "Reg Type": "",
            "Place of Supply": "Uttarakhand",
            "Country": "India",
            "Consignee State": "Uttarakhand",
            "Consignee GST": "",
        }
        new_rows.append(new_row)

    return new_rows


# Iterate over rows
for index, row in df1.iterrows():
    # print(str(row.iloc[0]), str(row.iloc[0]).startswith("A"))
    # If the row starts with a date (assuming the date is in the first cell)
    if pd.notna(row.iloc[0]):
        try:
            current_bill_date = pd.to_datetime(row.iloc[0], format="%d/%m/%Y").strftime(format= "%d/%m/%Y")
            continue
        except ValueError:
            # If the value is not a valid date, move to the next row
            pass

    # If the row starts with a bill number
    if str(row.iloc[0]).startswith("A"):
        # Set current bill number
        

        result_df = pd.concat(
            [
                
                result_df,
                pd.DataFrame(get_transformed_rows(row, bill_date=current_bill_date)),
            ],
            ignore_index=True,
        )

In [100]:
result_df.shape # rows, columns

(6219, 23)

In [101]:
# result_df.to_excel('transformed_bills.xlsx', index=False)

writer = pd.ExcelWriter('pandas_sale.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
result_df.to_excel(writer, sheet_name='Sheet1', index=False)

# Get the xlsxwriter objects from the dataframe writer object.
workbook  = writer.book
worksheet = writer.sheets['Sheet1']

In [102]:
worksheet.dim_rowmax, worksheet.dim_colmax

(6219, 22)

In [103]:
from openpyxl.utils import get_column_letter

amt_index = result_df.columns.get_loc("Amount") + 2

amt_col_letter = get_column_letter(amt_index)

print(f"{amt_col_letter = }")
print(f"{amt_index = }")


index_of_sheet = f'{amt_col_letter}{worksheet.dim_rowmax + 2}'
formula =  f'=SUM({amt_col_letter}2:{amt_col_letter}{worksheet.dim_rowmax + 1})'


worksheet.write_formula(index_of_sheet, formula)

writer.close()

amt_col_letter = 'J'
amt_index = 10
J6221 =SUM(J2:J6220)
